In [1]:
# ******ignore warnings
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import pandas as pd
from scipy import stats
import os


# ******files/data
from pydataset import data
import env
# import acquire
# import prepare


# ******visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# ******sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 01. Data Acquisition
### Acquiring telco data from Codeup DB

In [2]:
# function to connect to Codeup DB using credentials from env.py file 

def connect_to_db(db, user = env.user, host = env.host, password = env.password):
    '''
    This function takes in a database and login credentials to establish a connection to the
    Codeup database, returning a string that can be used to query the database in the read_sql function
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

### Reading querying the database (that we established a connection to in previous step), returning the database and a Dataframe

In [3]:
# variable to store query for read_sql
sql_query = '''select * from customers 
                join contract_types using (contract_type_id) 
                join internet_service_types using (internet_service_type_id) 
                join payment_types using (payment_type_id)'''

# variable to store returned database 
raw_data = pd.read_sql(sql_query ,connect_to_db('telco_churn'))
raw_data.head(2)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)


### Caching Database as .csv file

In [4]:
# writing newly acquired Dataframe to .csv file
raw_data.to_csv('telco_data.csv')

# Function for acquire.py file

In [5]:
# these two functions will be inputs for the main acquire_telco_data() function

def get_connection(db, user = env.user, host = env.host, password = env.password):
    '''
    This function takes in login credentials from env file as an arguments and will be used to establish a 
    connection to Codeup Database by returning a connection URL.
    
    This will be the first of two input functions for our final acquire_telco_data function.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

    
def new_telco_data():
    '''
    This function will store the sequel query that will be used, along with the get_connection function, 
    to read the telco data into a DataFrame, storing that DataFrame
    
    This will be the second of two input functions for our final acquire_telco_data function.
    ''' 
    
#   creating the variable that will hold the code used to query the Codeup database    
    sql_query = '''select * from customers 
                    join contract_types using (contract_type_id) 
                    join internet_service_types using (internet_service_type_id) 
                    join payment_types using (payment_type_id)'''
    
#   using the get_connection() function and sql_query to read in DataFrame from Codeup database
    df = pd.read_sql(sql_query, get_connection('telco_churn'))
    
    return df
    


In [6]:
def acquire_telco_data():
    '''
    This is the main acquire_telco_data function that checks to see if a local csv for telco data exists.
        If it does it will write the csv data into a DataFrame
            (else)
        If the telco data csv file does not exist locally, it will use the input function new_telco_data to 
        query the Codeup database and return the query as DataFrame .
            It will then cache the DataFrame and create local csv file. 
    '''
    if os.path.isfile('telco_data'):
#   if csv file already exists in local directory, this code will run

        df = pd.read_csv('telco_data', index_col = 0)
#       this line of code reads the telco csv file into a Database

    else:
#   if csv file does not exist in local directory, this code will run instead

        df = new_telco_data()
#       this line of code reads telco database into a DataFrame using the input function new_telco_data

        df.to_csv('telco_data.csv')
#       this line of code caches the telco data, creating a local csv file

    return df


In [7]:
# running final acquire_telco_data function and then previewing

raw_data = acquire_telco_data()
raw_data.head(2)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)


***

# 02 Data Preparation

In [8]:
# let's take a look at our raw_data df

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

***

## Let's look for redundant columns

In [9]:
# contract_type (object) and contract_type_id(int64)

pd.crosstab(raw_data.contract_type, raw_data.contract_type_id)

contract_type_id,1,2,3
contract_type,,,
Month-to-month,3875,0,0
One year,0,1473,0
Two year,0,0,1695


In [10]:
# we can drop contract_type, the dtypes are objects and only numerical dtypes can go into our model for later
# we will also create a variable to hold our prepare df ---> prepare_telco

prepare_telco = raw_data.drop(columns = 'contract_type')
prepare_telco.shape

(7043, 23)

In [11]:
# internet_service_type (object) and internet_service_type_id (int64)

pd.crosstab(prepare_telco.internet_service_type, prepare_telco.internet_service_type_id)

internet_service_type_id,1,2,3
internet_service_type,,,
DSL,2421,0,0
Fiber optic,0,3096,0
None,0,0,1526


In [12]:
# we can drop internet_service_type, the dtypes are objects and only numerical dtypes can go into our model 
# for later

prepare_telco = prepare_telco.drop(columns = 'internet_service_type')
prepare_telco.shape

(7043, 22)

In [13]:
# payment_type (object) and payment_type_id (int64)

pd.crosstab(prepare_telco.payment_type, prepare_telco.payment_type_id)

payment_type_id,1,2,3,4
payment_type,,,,
Bank transfer (automatic),0,0,1544,0
Credit card (automatic),0,0,0,1522
Electronic check,2365,0,0,0
Mailed check,0,1612,0,0


In [14]:
# we can drop internet_service_type, the dtypes are objects and only numerical dtypes can go into our model 
# for later

prepare_telco = prepare_telco.drop(columns = 'payment_type')
prepare_telco.shape

(7043, 21)

## Columns we dropped:
- contract_type (keeping contract_type_id)
- internet_service_type (keeping internet_service_type_id)
- payment_type (keeping payment_type_id)

DataFrame columns: from 24 ---> 21
### We can also use the crosstabs for mapping variables later

---

## Let's create a loop that prints the value_counts( ) for each column so we can know each column's unique values and associated totals

In [15]:
# figuring our code that will go into our for loop, testing it on one column

print(prepare_telco.payment_type_id.value_counts())

1    2365
2    1612
3    1544
4    1522
Name: payment_type_id, dtype: int64


In [16]:
# using value_counts() to get the unique values and their corresponding number of occurences for for loop

for col in prepare_telco.columns:
    print(f'Column: {col}')
    print(prepare_telco[col].value_counts())
    print('-----------------------------')
    print()


Column: payment_type_id
1    2365
2    1612
3    1544
4    1522
Name: payment_type_id, dtype: int64
-----------------------------

Column: internet_service_type_id
2    3096
1    2421
3    1526
Name: internet_service_type_id, dtype: int64
-----------------------------

Column: contract_type_id
1    3875
3    1695
2    1473
Name: contract_type_id, dtype: int64
-----------------------------

Column: customer_id
0689-NKYLF    1
9488-FVZCC    1
5458-CQJTA    1
7037-MTYVW    1
2455-USLMV    1
             ..
3813-DHBBB    1
5168-MQQCA    1
5651-WYIPH    1
1945-XISKS    1
3910-MRQOY    1
Name: customer_id, Length: 7043, dtype: int64
-----------------------------

Column: gender
Male      3555
Female    3488
Name: gender, dtype: int64
-----------------------------

Column: senior_citizen
0    5901
1    1142
Name: senior_citizen, dtype: int64
-----------------------------

Column: partner
No     3641
Yes    3402
Name: partner, dtype: int64
-----------------------------

Column: dependents
No  

***

## Let's map values to change object columns to numerical values

In [17]:
# creating a for loop that returns one list of our object columns and another with our numerical columns

object_cols = []
num_cols = []

for col in prepare_telco.columns:
    if prepare_telco[col].dtype == 'O':
        object_cols.append(col)
    else:
        num_cols.append(col)

In [18]:
# these (our object columns) are the columns we will need to map to numerical values or encode
print(f'There are {len(object_cols)} object columns.')
print('\n')
print(object_cols)

There are 15 object columns.


['customer_id', 'gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'total_charges', 'churn']


In [41]:
len(object_cols)

14

### total_charges column needs to be changed from object to float

In [19]:
# our total charges column has 11 rows that are email values
#     none of these customers churned

prepare_telco[prepare_telco.total_charges == ' ']

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
85,4,1,3,1371-DWPAZ,Female,0,Yes,Yes,0,No,...,Yes,Yes,Yes,Yes,Yes,No,No,56.05,,No
156,3,1,3,2775-SEFEE,Male,0,No,Yes,0,Yes,...,Yes,Yes,No,Yes,No,No,Yes,61.90,,No
236,2,1,3,4075-WKNIU,Female,0,Yes,Yes,0,Yes,...,No,Yes,Yes,Yes,Yes,No,No,73.35,,No
255,3,1,3,4472-LVYGI,Female,0,Yes,Yes,0,No,...,Yes,No,Yes,Yes,Yes,No,Yes,52.55,,No
339,2,1,3,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,...,Yes,Yes,Yes,No,Yes,Yes,No,80.85,,No
5681,2,3,3,2520-SGTTA,Female,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.00,,No
5717,2,3,3,3115-CZMZD,Male,0,No,Yes,0,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,,No
5727,2,3,3,3213-VVOLG,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.35,,No
5798,2,3,3,4367-NUYAO,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,25.75,,No
6007,2,3,3,7644-OMVMY,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,19.85,,No


In [20]:
# we will drop the rows where total_charges are blank by resaving the prepare DataFrame, 
#     using a Boolean mask to excluse the empty total_charges rows

prepare_telco = prepare_telco[prepare_telco.total_charges != ' ']
prepare_telco.shape

(7032, 21)

In [21]:
# now we can change the datatype in the total_charges column from object to float

prepare_telco.total_charges = prepare_telco.total_charges.astype(float)
prepare_telco.total_charges.dtype

dtype('float64')

In [22]:
# we will use .pop to remove total_charges from object_cols list

object_cols.pop(object_cols.index('total_charges'))

'total_charges'

In [23]:
print(f'There are now {len(object_cols)} object columns')
print('\n')
print(object_cols)

There are now 14 object columns


['customer_id', 'gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn']


## Mapping our binary, 'Yes', 'No' value variables:
0: No<br>
1: Yes
- churn
- dependents
- partner
- paperless_billing
- phone_service<br>


(<i>senior_citizen already 0, 1 values</i>)

In [24]:
# churn mapping
prepare_telco.churn = prepare_telco.churn.map({'No': 0, 'Yes': 1})
prepare_telco.churn.value_counts()

0    5163
1    1869
Name: churn, dtype: int64

In [25]:
# dependents mapping
prepare_telco.dependents = prepare_telco.dependents.map({'No': 0, 'Yes': 1})
prepare_telco.dependents.value_counts()

0    4933
1    2099
Name: dependents, dtype: int64

In [26]:
# partner mapping
prepare_telco.partner = prepare_telco.partner.map({'No': 0, 'Yes': 1})
prepare_telco.partner.value_counts()

0    3639
1    3393
Name: partner, dtype: int64

In [27]:
# paperless_billing mapping
prepare_telco.paperless_billing = prepare_telco.paperless_billing.map({'No': 0, 'Yes': 1})
prepare_telco.paperless_billing.value_counts()

1    4168
0    2864
Name: paperless_billing, dtype: int64

In [28]:
# phone_service mapping
prepare_telco.phone_service = prepare_telco.phone_service.map({'No': 0, 'Yes': 1})
prepare_telco.phone_service.value_counts()

1    6352
0     680
Name: phone_service, dtype: int64

## Encoding our Remaining Categorical Variables
- device_protection
    - No, Yes, No Internet Service
- multiple_lines	
    - Yes, No, No Phone Service
- online_backup	
    - Yes, No, No Phone Service
- online_security	
    - Yes, No, No Internet Service
- streaming_movies	
    - Yes, No, No Internet Service
- streaming_tv	
    - Yes, No, No Internet Service
- tech_support	
    - Yes, No, No Internet Service
- internet_service_type_id	
    - 1: DSL, 2: Fiber optic, 3: None
- payment_type_id	
    - 1: Electronic check, 2: Mailed check, 3: Bank transfer (automatic), 4: Credit card (automatic)
- contract_type_id	
    - 1: Month to Month, 2: One Year, 3: Two Year
- gender	
    - Male, Female

In [29]:
# created a list containing the name of each column to encode with dummy variable
dummy_cols = ['device_protection', 'multiple_lines', 'online_backup', 
                                         'online_security', 'streaming_movies', 'streaming_tv', 
                                         'tech_support', 'internet_service_type_id', 'payment_type_id', 
                                         'contract_type_id', 'gender']

In [30]:
# created new variable, dummy_df, to hold prepare_telco dataframe and "dummy"fied dummy column values
dummy_df = pd.get_dummies(prepare_telco, columns = dummy_cols)

# lowercased dummy column names
dummy_df.columns = dummy_df.columns.str.lower()

### Some of my dummy column names were not very clear (i.e. payment_type_1). I used a crosstab on the original raw_data DataFrame to map which values matched with the corresponding prefix numbers
- contract_type_id,
- payment_type_id, and
- internet_service_type

In [31]:
# first, I looked at contract_type
pd.crosstab(raw_data.contract_type_id, raw_data.contract_type)

contract_type,Month-to-month,One year,Two year
contract_type_id,,,
1,3875,0,0
2,0,1473,0
3,0,0,1695


In [32]:
#     contract_type_id_1   ---------> contract_mtm
#     contract_type_id_2   ---------> contract_one_yr
#     contract_type_id_3   ---------> contract_two_yr
dummy_df = dummy_df.rename(columns = 
               {'contract_type_id_1': 'dmy_contract_mtm',
               'contract_type_id_2': 'dmy_contract_one_yr',
               'contract_type_id_3': 'dmy_contract_two_yr'})

In [33]:
#     then payment_type
pd.crosstab(raw_data.payment_type_id, raw_data.payment_type)

payment_type,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
payment_type_id,,,,
1,0,0,2365,0
2,0,0,0,1612
3,1544,0,0,0
4,0,1522,0,0


In [34]:
#     payment_type_id_1   ---------> pay_elec_check
#     payment_type_id_2   ---------> pay_mail_check
#     payment_type_id_3   ---------> pay_bank_xfer_auto
#     payment_type_id_4   ---------> pay_credit_card_auto

dummy_df = dummy_df.rename(columns = {
                           'payment_type_id_1': 'dmy_pay_elec_check', 
                           'payment_type_id_2': 'dmy_pay_mail_check',
                           'payment_type_id_3': 'dmy_pay_bank_xfer_auto',
                           'payment_type_id_4': 'dmy_pay_credit_card_auto'})

In [35]:
#     and finally, internet_service_type
pd.crosstab(raw_data.internet_service_type_id, raw_data.internet_service_type)

internet_service_type,DSL,Fiber optic,None
internet_service_type_id,,,
1,2421,0,0
2,0,3096,0
3,0,0,1526


In [36]:
#     internet_service_type_1  ---->  DSL 
#     internet_service_type_2  ---->  Fiber Optic
#     internet_service_type_3  ---->  None

dummy_df = dummy_df.rename(columns = {
    'internet_service_type_id_1': 'internet_dsl', 
    'internet_service_type_id_2': 'internet_fiber',
    'internet_service_type_id_3': 'internet_none'
})

## Last minute cleanup before creating our prepare.py file with cleanup function(s)

In [37]:
# looking at the cleaned data info for any last minute cleanup before saving our prepare_telco, ready for 
#     exploration and modeling
dummy_df

,customer_id,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,...,internet_none,dmy_pay_elec_check,dmy_pay_mail_check,dmy_pay_bank_xfer_auto,dmy_pay_credit_card_auto,dmy_contract_mtm,dmy_contract_one_yr,dmy_contract_two_yr,gender_female,gender_male
0,0016-QLJIS,0,1,1,65,1,1,90.45,5957.90,0,...,0,0,1,0,0,0,0,1,1,0
1,0017-DINOC,0,0,0,54,0,0,45.20,2460.55,0,...,0,0,0,0,1,0,0,1,0,1
2,0019-GFNTW,0,0,0,56,0,0,45.05,2560.10,0,...,0,0,0,1,0,0,0,1,1,0
3,0056-EPFBG,0,1,1,20,0,1,39.40,825.40,0,...,0,0,0,0,1,0,0,1,0,1
4,0078-XZMHT,0,1,0,72,1,1,85.15,6316.20,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9962-BFPDU,0,1,1,1,1,0,20.05,20.05,0,...,1,0,1,0,0,1,0,0,1,0
7039,9967-ATRFS,0,0,0,19,1,0,19.90,367.55,0,...,1,0,1,0,0,1,0,0,1,0
7040,9970-QBCDA,0,0,0,6,1,0,19.70,129.55,0,...,1,0,0,0,1,1,0,0,1,0
7041,9975-SKRNR,0,0,0,1,1,0,18.90,18.90,0,...,1,0,1,0,0,1,0,0,0,1


In [38]:
# last step of cleanup ----> dropping customer_id column
prepare_telco = dummy_df.drop(columns = ['customer_id'])

In [40]:
# DataFrame cleaned and prep'd for exploration and modeling :)
prepare_telco

,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,device_protection_no,...,internet_none,dmy_pay_elec_check,dmy_pay_mail_check,dmy_pay_bank_xfer_auto,dmy_pay_credit_card_auto,dmy_contract_mtm,dmy_contract_one_yr,dmy_contract_two_yr,gender_female,gender_male
0,0,1,1,65,1,1,90.45,5957.90,0,0,...,0,0,1,0,0,0,0,1,1,0
1,0,0,0,54,0,0,45.20,2460.55,0,1,...,0,0,0,0,1,0,0,1,0,1
2,0,0,0,56,0,0,45.05,2560.10,0,0,...,0,0,0,1,0,0,0,1,1,0
3,0,1,1,20,0,1,39.40,825.40,0,0,...,0,0,0,0,1,0,0,1,0,1
4,0,1,0,72,1,1,85.15,6316.20,0,0,...,0,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,1,1,0,20.05,20.05,0,0,...,1,0,1,0,0,1,0,0,1,0
7039,0,0,0,19,1,0,19.90,367.55,0,0,...,1,0,1,0,0,1,0,0,1,0
7040,0,0,0,6,1,0,19.70,129.55,0,0,...,1,0,0,0,1,1,0,0,1,0
7041,0,0,0,1,1,0,18.90,18.90,0,0,...,1,0,1,0,0,1,0,0,0,1
